In [1]:
import torch
x_dummy = torch.randn(5, 2)

## SIREN and FINER MODEL

In [2]:
from models.implicit_neural_representations.inr_models.siren_model import SirenModel, FinerModel

In [3]:
# initialize siren model without residual layers
siren_model = SirenModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    first_omega_0=30.0,
    hidden_omega_0=30.0,
    residual_net=False
)

In [4]:
output = siren_model(x_dummy)

In [5]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[-0.0022,  0.0074],
        [ 0.0101, -0.0041],
        [ 0.0097,  0.0455],
        [ 0.0082, -0.0170],
        [ 0.0322,  0.0173]], grad_fn=<MmBackward0>)


In [6]:
finer_model = FinerModel(
    in_features=2,
    out_features=2,
    bias=True,
    hidden_layers=5,
    hidden_features=128,
    first_omega_0=30.,
    hidden_omega_0=30.,
    first_bias=10,
    hidden_bias=10,
    residual_net=False
)

In [7]:
output = finer_model(x_dummy)

In [8]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.0022, -0.0383],
        [ 0.0020, -0.0389],
        [ 0.0021, -0.0379],
        [ 0.0024, -0.0384],
        [ 0.0022, -0.0386]], grad_fn=<MmBackward0>)


In [9]:
siren_residual_model = SirenModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    first_omega_0=30.0,
    hidden_omega_0=30.0,
    residual_net=True
)

In [10]:
output = siren_residual_model(x_dummy)

In [11]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.0292,  0.1227],
        [ 0.1023,  0.1233],
        [ 0.0185,  0.1181],
        [-0.0660, -0.0753],
        [ 0.0515,  0.0200]], grad_fn=<MmBackward0>)


In [12]:
finer_residual_model = FinerModel( 
    in_features=2,
    out_features=2,
    bias=True,
    hidden_layers=5,
    hidden_features=128,
    first_omega_0=30.,
    hidden_omega_0=30.,
    first_bias=10,
    hidden_bias=10,
    residual_net=True
)

In [13]:
output = finer_residual_model(x_dummy)

In [14]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.0607, -0.0556],
        [ 0.1394, -0.0182],
        [-0.0167,  0.0206],
        [ 0.0109, -0.0044],
        [-0.0342, -0.0734]], grad_fn=<MmBackward0>)


## Gaussian Model and Finer Gaussian

In [15]:
from models.implicit_neural_representations.inr_models.gaussian_model import GaussianModel, GaussianFinerModel

In [16]:
# initialize gaussian model without residual layers
gaussian_model = GaussianModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    scale=30.0,
    weight_init=0.1,
    residual_net=False
)

In [17]:
output = gaussian_model(x_dummy)

In [18]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.4340, -0.0834],
        [ 0.4501,  0.0324],
        [ 0.2990,  0.5150],
        [ 0.1682, -0.3585],
        [ 0.1744, -0.4943]], grad_fn=<MmBackward0>)


In [19]:
gaussian_residual_model = GaussianModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    scale=30.0,
    weight_init=0.1,
    residual_net=True
)

In [20]:
output = gaussian_residual_model(x_dummy)

In [21]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.7280,  0.7350],
        [ 0.0312,  0.9094],
        [ 0.8563,  1.4003],
        [ 0.8930,  0.4530],
        [-0.2096,  0.5700]], grad_fn=<MmBackward0>)


In [23]:
gaussian_finer_model = GaussianFinerModel(
    in_features=2,
    out_features=2,
    scale=30.0,
    omega_f=2.5,
    bias_init=5.0,
    weight_init=0.1,
    residual_net=False
)

In [24]:
output = gaussian_finer_model(x_dummy)

In [25]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.1229, -0.2867],
        [ 0.1201, -0.1480],
        [ 0.2647, -0.1398],
        [-0.0608, -0.0166],
        [ 0.0929, -0.1008]], grad_fn=<MmBackward0>)


In [26]:
gaussian_finer_residual_model = GaussianFinerModel(
    in_features=2,
    out_features=2,
    scale=30.0,
    omega_f=2.5,
    bias_init=5.0,
    weight_init=0.1,
    residual_net=True
)

In [27]:
output = gaussian_finer_residual_model(x_dummy)

In [28]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[-1.2238,  1.1677],
        [-0.3147,  0.9046],
        [-0.2586, -0.3755],
        [ 0.3574,  0.7538],
        [ 0.8575, -0.3224]])
Output:
tensor([[ 0.0819,  0.2632],
        [ 0.1635, -0.2622],
        [ 0.2216,  0.1712],
        [-0.2419, -0.0927],
        [-0.3844, -0.0591]], grad_fn=<MmBackward0>)
